<a href="https://colab.research.google.com/github/ssv273/Neural_Univesity/blob/main/hw_32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyldavis

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 11.3 MB 20.6 MB/s 
     |████████████████████████████████| 15.7 MB 128 kB/s 
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=799623a33fde4333b75e1f9648a4f9053fa46ee5db0e0d8596b386b1bb791165
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyldavis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take int

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk
import pandas as pd

from google.colab import drive

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Neural_Univesity/hw_32/ner.csv', encoding = "ISO-8859-1", error_bad_lines=False, index_col=0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 281837: expected 25 fields, saw 34\n'


In [ ]:
df = df.drop_duplicates()

In [ ]:
df.head()

,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,pos,prev-iob,prev-lemma,prev-pos,prev-prev-iob,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,NNS,__START1__,__start1__,__START1__,__START2__,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,IN,O,thousand,NNS,__START1__,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,NNS,O,of,IN,O,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,VBP,O,demonstr,NNS,O,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,march,through,london,NNP,capitalized,London,IN,lowercase,through,VBN,O,have,VBP,O,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [ ]:
df = df[['sentence_idx', 'word','tag']] #берем только необходимые столбцы

In [ ]:
df.head(25)

,sentence_idx,word,tag
0,1.0,Thousands,O
1,1.0,of,O
2,1.0,demonstrators,O
3,1.0,have,O
4,1.0,marched,O
5,1.0,through,O
6,1.0,London,B-geo
7,1.0,to,O
8,1.0,protest,O
9,1.0,the,O


In [ ]:
max_idx = int(df.sentence_idx.max())
max_idx

47959

In [ ]:
df.loc[df['sentence_idx']==1, 'word']

0         Thousands
1                of
2     demonstrators
3              have
4           marched
5           through
6            London
7                to
8           protest
9               the
10              war
11               in
12             Iraq
13              and
14           demand
15              the
16       withdrawal
17               of
18          British
19           troops
20             from
21             that
22          country
23                .
Name: word, dtype: object

In [ ]:
sequences = []
for i in range(1, max_idx+1): #проходимся по всем id предложений
  if len(df.loc[df['sentence_idx']==i, 'word'].values.tolist()) > 3:
    sequences.append(df.loc[df['sentence_idx']==i, 'word'].values.tolist()) #делаем список из слов с одним id (предложения)

In [ ]:
len(sequences)

35168

In [ ]:
print(sequences[2])

['They', 'marched', 'from', 'the', 'Houses', 'of', 'Parliament', 'to', 'a', 'rally', 'in', 'Hyde', 'Park', '.']


In [ ]:
np.random.seed(42)
x = []
for n, i in enumerate(np.random.randint(0, len(sequences), 10)):
    x.append(sequences.pop(i - n))

In [ ]:
len(sequences)

35158

In [ ]:
sequences_ = [' '.join(sequence) for sequence in sequences]
x_sequences = [' '.join(sequence) for sequence in x]

In [ ]:
x_sequences

['China has reported two other bird flu deaths this year .',
 'Officials said Sunday that the group named KindHearts has coordinated with Hamas leaders and has ties to two other U.S.-based charities accused of links to terrorism .',
 "Morocco 's long-term challenges include improving education and job prospects for young Moroccans , closing the disparity in wealth between the rich and the poor , confronting corruption , and expanding and diversifying exports beyond phosphates and low-value-added products .",
 'In 2004 , about 170 people were killed .',
 'Sales , however , were little changed at 2.46 billion guilders , compared with 2.42 billion guilders .',
 'An Israeli newspaper reports that Prime Minister Ehud Olmert is urging the United States to impose a naval blockade on Iran in an effort to curb its controversial nuclear program .',
 'Under the regulations , foreign news and music broadcasts must be edited locally .',
 "Zambia 's deputy leader says President Levy Mwanawasa has un

In [ ]:
sequences_[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [ ]:
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel

In [ ]:
common_dictionary = Dictionary(sequences) #подаем список списков слов, формируем слова
common_corpus = [common_dictionary.doc2bow(text) for text in sequences] #превращаем предложение в вектор bow

lda = LdaModel(common_corpus, num_topics=10) #тренируем модель

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [ ]:
vis_data = gensimvis.prepare(lda, common_corpus, common_dictionary) #подготовим объет визуальных данных по пространству топиков
pyLDAvis.display(vis_data) #визуализируем данные

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
def format_topics_sentences(ldamodel=lda, corpus=common_corpus, texts=sequences_):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda, corpus=common_corpus, texts=sequences_)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,1.0,0.5398,"16, 13, 0, 11, 85, 5, 47, 18, 67, 10",Thousands of demonstrators have marched throug...
1,1,1.0,0.5773,"16, 13, 0, 11, 85, 5, 47, 18, 67, 10",Families of soldiers killed in the conflict jo...
2,2,1.0,0.9400,"16, 13, 0, 11, 85, 5, 47, 18, 67, 10",They marched from the Houses of Parliament to ...
3,3,1.0,0.5654,"16, 13, 0, 11, 85, 5, 47, 18, 67, 10","Police put the number of marchers at 10,000 wh..."
4,4,1.0,0.5958,"16, 13, 0, 11, 85, 5, 47, 18, 67, 10",The protest comes on the eve of the annual con...
5,5,1.0,0.7097,"16, 13, 0, 11, 85, 5, 47, 18, 67, 10",The party is divided over Britain 's participa...
6,6,6.0,0.5876,"0, 11, 16, 47, 85, 13, 35, 5, 134, 199",The London march came ahead of anti-war protes...
7,7,8.0,0.5327,"16, 0, 13, 18, 47, 85, 61, 11, 221, 159",The International Atomic Energy Agency is to h...
8,8,1.0,0.4317,"16, 13, 0, 11, 85, 5, 47, 18, 67, 10",Iran this week restarted parts of the conversi...
9,9,4.0,0.6311,"16, 85, 0, 18, 5, 13, 11, 47, 67, 221",Iranian officials say they expect to get acces...


In [ ]:
x_common_dictionary = Dictionary(x) #подаем список списков слов, формируем слова
x_common_corpus = [common_dictionary.doc2bow(text) for text in x] #превращаем предложение в вектор bow
x_df_topic_sents_keywords = format_topics_sentences(ldamodel=lda, corpus=x_common_corpus, texts=x_sequences)

# Format
x_df_dominant_topic = x_df_topic_sents_keywords.reset_index()
x_df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
x_df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,1.0,0.9250,"16, 13, 0, 11, 85, 5, 47, 18, 67, 10",China has reported two other bird flu deaths t...
1,1,8.0,0.4390,"16, 0, 13, 18, 47, 85, 61, 11, 221, 159",Officials said Sunday that the group named Kin...
2,2,4.0,0.4385,"16, 85, 0, 18, 5, 13, 11, 47, 67, 221",Morocco 's long-term challenges include improv...
3,3,6.0,0.7216,"0, 11, 16, 47, 85, 13, 35, 5, 134, 199","In 2004 , about 170 people were killed ."
4,4,3.0,0.3288,"0, 18, 16, 85, 5, 67, 47, 221, 13, 72","Sales , however , were little changed at 2.46 ..."
5,5,7.0,0.5942,"0, 18, 16, 182, 15, 278, 47, 81, 574, 61",An Israeli newspaper reports that Prime Minist...
6,6,4.0,0.4845,"16, 85, 0, 18, 5, 13, 11, 47, 67, 221","Under the regulations , foreign news and music..."
7,7,8.0,0.5951,"16, 0, 13, 18, 47, 85, 61, 11, 221, 159",Zambia 's deputy leader says President Levy Mw...
8,8,1.0,0.6586,"16, 13, 0, 11, 85, 5, 47, 18, 67, 10",The probe of the area where two pieces of ocea...
9,9,6.0,0.9098,"0, 11, 16, 47, 85, 13, 35, 5, 134, 199",Two car bomb blasts near the Shi'ite shrine ci...


update LDA model

In [ ]:
lda.update(x_common_corpus)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [ ]:
x_df_topic_sents_keywords = format_topics_sentences(ldamodel=lda, corpus=x_common_corpus, texts=x_sequences)

# Format
x_df_dominant_topic = x_df_topic_sents_keywords.reset_index()
x_df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
x_df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,1.0,0.9250,"16, 13, 0, 11, 85, 5, 47, 18, 67, 10",China has reported two other bird flu deaths t...
1,1,8.0,0.5985,"16, 0, 13, 18, 61, 47, 11, 85, 159, 5",Officials said Sunday that the group named Kin...
2,2,4.0,0.4516,"16, 85, 0, 18, 5, 11, 13, 47, 221, 67",Morocco 's long-term challenges include improv...
3,3,6.0,0.7230,"0, 11, 16, 47, 85, 35, 13, 5, 199, 134","In 2004 , about 170 people were killed ."
4,4,3.0,0.3889,"0, 18, 16, 85, 5, 67, 47, 221, 13, 199","Sales , however , were little changed at 2.46 ..."
5,5,7.0,0.6316,"0, 18, 16, 182, 15, 278, 47, 81, 574, 102",An Israeli newspaper reports that Prime Minist...
6,6,4.0,0.5303,"16, 85, 0, 18, 5, 11, 13, 47, 221, 67","Under the regulations , foreign news and music..."
7,7,8.0,0.6026,"16, 0, 13, 18, 61, 47, 11, 85, 159, 5",Zambia 's deputy leader says President Levy Mw...
8,8,1.0,0.7048,"16, 13, 0, 11, 85, 5, 47, 18, 67, 10",The probe of the area where two pieces of ocea...
9,9,6.0,0.9144,"0, 11, 16, 47, 85, 35, 13, 5, 199, 134",Two car bomb blasts near the Shi'ite shrine ci...


вероятности принадлежности топиков к определенной теме увеличились.

